# Mel Spectrogram and MFCCs

Alexandre R.J. Francois

In [ ]:
import numpy as np
from matplotlib import colormaps as mcm
import matplotlib.pyplot as plt
import librosa
import soundfile as sf
from IPython.display import Audio 

from noFFT import resonate

In [ ]:
#https://librosa.org/doc/main/recordings.html


# y, sr = sf.read(librosa.ex("brahms"), frames=500000)
# y, sr = sf.read(librosa.ex("vibeace"), frames=500000)
# y, sr = sf.read(librosa.ex("sweetwaltz"))
y, sr = sf.read(librosa.ex("libri1"))
# y, sr = sf.read(librosa.ex("libri2"))
# y, sr = sf.read(librosa.ex("libri3"))
# y, sr = sf.read(librosa.ex("robin"))

print(sr)

librosa.display.waveshow(y, sr=sr)
print(y.shape)
print(y.dtype)

float_y = np.array(y, dtype=np.float32)
print(float_y.dtype)

Audio(data=y, rate=sr)


In [ ]:
# Mel frequencies

fmin = 0
fmax = 8000.0
n_mels=128

freqs = librosa.mel_frequencies(n_mels=n_mels, fmin=fmin, fmax=fmax, htk=False)
freqs[0] = freqs[1]
# print(freqs)

alphas = 1 - np.exp(-(1 / sr) * freqs / np.log10(1+freqs))
alphas[np.isnan(alphas)] = 0
# print(alphas)

hop_length = 1
dhl = 32


In [ ]:
from noFFT_utils import resonate_wrapper, frequency_sweep

Rcx = resonate_wrapper(y=y, sr=sr, frequencies=freqs, alphas=alphas, hop_length=hop_length, output_type='complex')
Rcx = Rcx.T
# print(Rcx.shape, Rcx.dtype)

# compute powers
R_pows = np.abs(Rcx) ** 2
# print(R_pows.shape, R_pows.dtype)

# Equalize
eq = frequency_sweep(frequencies=freqs, alphas=alphas, sr=sr)
Req_pows = (eq * R_pows.T).T

R_db = librosa.power_to_db(Req_pows, ref=np.max)
# print(R_db[:, ::dhl].shape)

S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000, hop_length=dhl)
# print(S.shape)
S_db = librosa.power_to_db(S,ref=np.max)

fig, ax = plt.subplots(nrows=2, ncols=2, sharex='col', figsize=(8, 6), dpi=300)
img = librosa.display.specshow(
    S_db,
    sr=sr,
    hop_length=dhl,
    x_axis='s',
    y_axis='mel',
    fmax=fmax,
    ax=ax[0][0])
ax[0][0].set(title='Librosa Mel spectrogram')
ax[0][0].label_outer()
img = librosa.display.specshow(
    S_db[:,300:800],
    sr=sr,
    hop_length=dhl,
    x_axis='s',
    y_axis='mel',
    fmax=fmax,
    ax=ax[0][1])
ax[0][1].set(title='Librosa (detail)')
ax[0][1].label_outer()

img = librosa.display.specshow(
    R_db[:, ::dhl],
    sr=sr,
    hop_length=dhl,
    x_axis="s",
    y_axis="mel",
    fmax=fmax,
    ax=ax[1][0],
)
ax[1][0].set(title="Resonate Mel spectrogram")
ax[1][1].label_outer()
img = librosa.display.specshow(
    R_db[:, 9600:25600:dhl],
    sr=sr,
    hop_length=dhl,
    x_axis="s",
    y_axis="mel",
    fmax=fmax,
    ax=ax[1][1],
)
ax[1][1].set(title="Resonate (detail)")
fig.colorbar(img, ax=ax, format="%+2.f dB")


In [ ]:
# Librosa MFCC

mfccs = librosa.feature.mfcc(S=S_db, n_mfcc=21)
# mfccs = librosa.feature.mfcc(y=y, sr=sr)
print(mfccs.shape)

mfccs_r = librosa.feature.mfcc(S=R_db[:, ::dhl], ref=np.max, n_mfcc=21)
print(mfccs_r.shape)

fig, ax = plt.subplots(nrows=2, ncols=2, sharex='col', figsize=(8, 6), dpi=300)
img = librosa.display.specshow(
    mfccs[1:],
    hop_length=dhl,
    x_axis='s',
    ax=ax[0][0])
ax[0][0].set(title='Librosa MFCCs')
ax[0][0].label_outer()
img = librosa.display.specshow(
    mfccs[1:, 300:800],
    hop_length=dhl,
    x_axis='s',
    ax=ax[0][1])
ax[0][1].set(title='Librosa MFCCs (detail)')
ax[0][1].label_outer()
img = librosa.display.specshow(
    mfccs_r[1:],
    hop_length=dhl,
    x_axis='s',
    ax=ax[1][0])
ax[1][0].set(title='Resonate MFCCs')
ax[1][0].label_outer()
img = librosa.display.specshow(
    mfccs_r[1:, 300:800],
    hop_length=dhl,
    x_axis='s',
    ax=ax[1][1])
ax[1][1].set(title='Resonate MFCCs (detail)')
fig.colorbar(img, ax=ax, format="%+2.f dB")





In [ ]:

fig, ax = plt.subplots(nrows=2, sharex=True, figsize=(8, 8), dpi=300)
img = librosa.display.specshow(mfccs[1:], hop_length=dhl, x_axis='s', ax=ax[0])
fig.colorbar(img, ax=[ax[0]])
ax[0].set(title='Librosa MFCC')
ax[0].label_outer()
img = librosa.display.specshow(mfccs_r[1:], hop_length=dhl, x_axis='s', ax=ax[1])
fig.colorbar(img, ax=[ax[1]])
ax[1].set(title='Resonate MFCC')

In [ ]:
fig, ax = plt.subplots(nrows=2, sharex=True, figsize=(8, 8), dpi=300)
img = librosa.display.specshow(mfccs[1:, 300:800], hop_length=dhl, x_axis='s', ax=ax[0])
fig.colorbar(img, ax=[ax[0]])
ax[0].set(title='Librosa MFCC (detail)')
ax[0].label_outer()
img = librosa.display.specshow(mfccs_r[1:, 300:800], hop_length=dhl, x_axis='s', ax=ax[1])
fig.colorbar(img, ax=[ax[1]])
ax[1].set(title='Resonate MFCC (detail)')

In [ ]:
from matplotlib import colormaps as mcm
# list(colormaps)
magma = mcm['magma']
print(magma.colors)
# coolwarm = colormaps['coolwarm']
# print(coolwarm.colors)


# get colormap values for Spectrogram

map = mcm["coolwarm"].resampled(256)

colors = map(range(256))

# intmap = np.round(map.colors[:, 0:3] * 255).astype(int)

intmap = np.round(colors[:, 0:3] * 255).astype(int)

np.savetxt(
    "coolwarm_colormap_int.txt",
    intmap,
    fmt="%d",
    delimiter=", ",
    newline="],\n[",
)




